In [1]:
from ipywidgets import interactive, widgets, fixed
from surfer import Brain as surface
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import minmax_scale

import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# spectrome imports
from spectrome.brain import Brain
from spectrome.utils import functions, path

In [2]:
%gui qt

# set up Pysurfer variables
subject_id = "fsaverage"
hemi = ["lh","rh"]
surf = "white"

"""
Read in the automatic parcellation of sulci and gyri.
"""

hemi_side = "lh"
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi_side + ".aparc.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

In [3]:
# function for viewing canonical networks:
def get_fc_values(fc_df, labels, fc_name):
    # get our data ready in both hemispheres
    fc_network = fc_df.loc[fc_name].values
    lh_cort = minmax_scale(fc_network[0:34])
    rh_cort = minmax_scale(fc_network[34:68])

    # for pysurfer requirements
    lh_pad = np.insert(lh_cort, [0, 3], [0, 0])
    rh_pad = np.insert(rh_cort, [0, 3], [0, 0])

    lh_fc = lh_pad[labels]
    rh_fc = rh_pad[labels]

    fc_brain = surface(
        subject_id,
        "both",
        surf,
        background="white",
        alpha=1,
        title="Canonical Networks",
    )
    fc_brain.hide_colorbar()
    fc_brain.add_data(lh_fc, hemi="lh", thresh=0.15, colormap=plt.cm.autumn_r, remove_existing=True)
    fc_brain.add_data(rh_fc, hemi="rh", thresh=0.15, colormap=plt.cm.autumn_r, remove_existing=True)
    #fc_brain.scale_data_colormap(color_fmin, color_fmid, color_fmax, transparent=False)
    return lh_fc, rh_fc

In [4]:
fc_names = [
    "Visual",
    "Limbic",
    "Default",
    "Somatomotor",
    "Frontoparietal",
    "Ventral_Attention",
    "Dorsal_Attention",
]
# Load Pablo's canonical networks in DK atlas:
fc_dk = np.load("/Users/xxie/lab/complex_laplacian/data/com_dk.npy", allow_pickle=True).item()
fc_dk_normalized = pd.read_csv("/Users/xxie/lab/complex_laplacian/data/DK_dictionary_normalized.csv").set_index(
    "Unnamed: 0"
)

<p style = "text-align:center; font-size:200%;">How do functional brain networks emerge from the underlying wiring of the brain</p>
<p style = "text-align:center; font-size:120%;"><em>Frontiers in Neuropsychiatry Seminars</em></p>

<center>
<img src = "../assets/tract_viz.gif" alt = "intro" width = "400" style = "align:center"/>
</center>

<p style = "text-align:center; font-size:110%;">Xi He Xie</p>
<p style = "text-align:center; font-size:110%;">Department of Neuroscience, Weill Cornell Medicine</p>
<p style = "text-align:center; font-size:110%;"> Visiting Graduate, University of California San Francisco </p>
<p style = "text-align:center; font-size:110%;">October 13th, 2020</p>

What's so special about the human brain?
---

 - Humans have the remarkable ability to invent symbol systems such as the arabic numeric system or the alphabet.

<center>
    <img src = "../assets/lost_data.png" alt = "messy_data" width = "1200" style = "align:center" />
</center>

<aside class="notes">
Before I start talking in jargons and data, let's think for a second. I'm sure everyone here had days like this guy, we are constantly reading papers, crunching data, doing numerical analysis, and it can get extremely messy if you don't practice good organizational routines. Despite all that, our brains are still efficiently taking these symbols, putting them into context, and processes the complicated information... if you had enough coffee
</aside>

#### Exercise:

 - What is so special about human brain functionality that allows it to expand its functionality to acquire these cultural tools? (Language, reading, arithmetics) Let's examine the two possible hypothesis:

 1) Cultural functionality is acquired by expanded cortical plasticity unique to humans.
     

2) Humans have evolved new specialized circuitry, each providing new cognitive functions.

<aside class="notes">
On one extreme, you can have (1), on the other you can have (2). So very quickly, by a show of hands, who agrees with 1? la la la... who agrees with (2)?
    
 
</aside>

 - Human brians accomodate a broad range of new functions through learning.
 - At the extreme, the brain's architecture exerts little to no contraints on the range of competence we can acquire.
 - This may also suggest individual brain's implementation of cultural abilities is highly variable across individuals.

 - Humans experienced evolutionary pressure to develop brain circuitry for understanding symbols?
 - But reading was invented ~5000 years ago, and artihmetics even more recent!

 - Primates have prior brain functions and structures that were eventually adapted and efficiently used by humans.
 
<center>
    <img src = "../assets/xkcd_surgery.png" alt = "xkcd" width = "1200" style = "align:center" />
<center/>
<aside class="notes">
So this is from xkcd, and it's true there's an xkcd comic for everything. The brain's structure itself stays the same, but when our ancestors acquire new tools like symbols in arithmetics and reading, their brains adapted what was already available in primate brains and expand it's capabilities like a software update.
</aside>

### The structure-function problem in neuroscience:
The brain's structure at various spatial scales is critical for determining its function.

<b>My thesis' interest in particular: Finding the relationship between the brain's <em>structural wiring</em> and the <em>functional</em> patterns of neural activity.</b>
 - Consider the brain to be a macroscopic network of nodes and edges:
     - Identifiable gray matter brain regions of interest (ROIs)
     - Long range connections between brain regions via white matter fiber bundles

<center>
    <img src = "../assets/brain_nodes_edges.png" alt = "braingraph" width = "400" style = "align:right"/>
</center>

#### We can measure structure and function non-invasively in humans with ...

### Structural data: diffusion weighted imaging
<center>
    <img src = "../assets/tract_viz.gif" alt = "dti" width = "1200" style = "align:center"/>
</center>

#### Quantify all the traced white matter fiber tracts between all brain regions into a matrix (Structural connectivity)

<center>
<img src = "../assets/SC-example.png" alt = "SCmatrix" width = "1000" style = "align:center"/>
</center>
<aside class="notes">
Now we align both the white matter fibers and our anatomical MRI scans, from each of these ROIs, we can start counting the number of fibers going to each of the other connected regions, and summarize all the connections in this connectivity matrix. The brain is what we like to call a small world network, where there are dense local connections for specialized function, and long range connections for integration of function with other hubs of connections. Just the inter- and intra- hemispheric connections you see here, dense within hemisphere, sparse across hemispheres.
</aside>

# What about functional data?
### 1) Human Magnetoencephalography (MEG) frequency spectrum:
<center>
    <img src = "../assets/meg_spectrum_example.png" alt = "actualmeg" width = "1200" style = "align:center"/>
</center>
<aside class="notes">
The first functional human brain data we collect is called MEG. Where we have our subjects sit inside an expensive insulated room under an even more expensive helmet. The helmet is cooled to near absolute zero to cool down the SQUIDS (superconducting quantum interference devices), these sensors can detect subtle changes in magnetic field as induced by evoked currents from the neurons. Anyone remember the right hand rule from physics class?
    
The result from each sensor is then mapped to our brain regions of interest, but we don't like looking at it over time, because all we would see are squiggly oscillatory lines. So we decompose these oscillations into combinations of sin/cos waves of various frequencies and power, and plot them here in what we call a power spectrum. Humans have very replicable and similar power spectrums, where we have idenfiable delta, theta, alpha, beta, gamma waves. The alpha wave usually stands out the most, if everyone closed their eyse right now, the 10 Hz signal in the posterior regions of your brain would just light up! And beta band is often mentioned in motor tasks with beta suppresion at the lateral motor cortex
</aside>

<center>
    <img src = "../ACCESS2020/images/fmri_modalities.png" alt="fmri" width="1300" style="align:center" />
</center>

<aside class="notes">
Structural: The hydrogen molecules in our bodies contains 1 positive charge (protons), like the earth spinning on its axis with magnetic poles, each spinning charged molecule is also like a tiny magnet that spins around it's own axis. MRI uses magnetic fields and radio waves to measure locations of these molecules. These protons are all in random directions, the MRI machine uses a magnet that ranges from 1-7 Teslas to align all our protons in specific directions. For reference, the earth's magnetic field is ~0.6e-4T. The scanners produce a magnetic field that's 60,000 times stronger! 
fMRI: In our brains, we have billions of neurons, when a lot of these neurons begin to fire, the body rushes in oxygen to help supply energy. This is called a hemodynamic response, and what we measure is called blood oxygen level dependent signal (BOLD). 
DWI: The brain is connected by long distance projections called axonal fiber tracts. We specifically track the movement of water molecules inside the brain. And build a gradient of water diffusion. 
</aside>

2) Canonical Functional Networks
---
 - Obtained from resting-state fMRI recordings
 - Consistently observed functional networks in both resting-state fMRI and MEG
 - There are seven of these prominent, most frequently visited functional networks identified by neuroimaging studies.

In [5]:
interactive(
    get_fc_values,
    fc_df=fixed(fc_dk_normalized),
    labels=fixed(labels),
    fc_name=widgets.RadioButtons(
        options=fc_names, value="Limbic", description="Select canonical network"
    ),
)

interactive(children=(RadioButtons(description='Select canonical network', index=1, options=('Visual', 'Limbic…

## Measuring brain structure and function:

We have ...
- Resting-state functional recordings from MEG, fMRI functional networks

 - Anatomical magnetic resonance images (MRIs) to co-register MEG coordinates to "head" coordinates

 - Parcellated brain regions from MRI (68 cortical regions and 18 subcortical regions in our atlas of choice)

 - Diffusion weighted imaging (DTI) for anatomical wiring architecture via tractography

 - Hard working graduate students!

## Solving the structure-function problem: what's been done?

- fMRI, sensor and source space MEG/EEG modeling.

<center>
    <img src = "../assets/conventional_NMM.png" alt = "NMM" width = "600" style = "align:center"/>
</center>

 - complex nonlinear simulations of whole brain activity combined with structural connectivity
     - Too many parameters, interpretation is difficult.
     - Has not been evaluated whether their models capture spatial patterns of activity.

### Questions we aim to address:

<center><b> 1 - Can we extract canonical functional networks from structural networks?</b></center>

<center> <b> 2 - Can we reproduce empirical MEG frequency spectrum given a structural network? </b> </center>

<center><b> Can our model be simple? </b></center>
<center> hierarchical: same model for microscopic and macroscopic network transmission </center>
<center> low dimensional: small number of global model parameters </center>
<center> analytical: no numerical simulations preferable</center>

### How do canonical functional networks arise from structural networks?

We have a new framework combining connectivity matrices and distance matrices that results in a complex structural connectome (Laplacian), on which we do an eigen decomposition analysis with only 2 global parameters, coupling strength $\alpha$ and wave number $k$:
<center>
<img src = "images/fig1_overview_v3.png" alt = "workflow" width = "1800" style = "align:center"/>
</center>
<aside class="notes">
READ THE SLIDE

Remember the rotating tractogram I showed before? And we extracted structural connetivitty matrix by counting white matter connections? That's the yellow and red matrix here. But we also want to obtain the distance of all our tracts and summarize them the same way in this blue and white matrix, calling it the distance adjacency matrix. The upper triangle here is scaled by alpha, the global coupling term, indicating how much of our signal comes from connections rather than local dynamics, the second parameter wave number is paired with the lower triangle, the wave number is a spatial frequency describing the amount of distance a signal in the brain travels per oscillation. With these 2 parameters, we can obtain this next matrix called the complex Laplacian. This matrix describes the characteristic signal spread patterns in a network with eigenmodes. Now imagine you are a signal in the brain, when you travel from region to region, it's not random, you must follow the brain's structural wiring, and this matrix tells you how these wirings would guide your signal. And if we project these patterns onto the brains, we can start comparing these eigenmodes to our functional networks!
</aside>

Our goal is to relate the eigenmodes of the complex structural connectome to the canonical functional networks.

Delays become phases in the frequency domain, so we define a complex connectivity matrix as a function of $\omega$: $\pmb{C}^*(\omega) = \{ c_{j,k} exp(-j\omega \pmb{\tau_{j,k}^v}) \}$, then a normalized complex connectivity matrix at any given $\omega$ is:
$$ \pmb{C}(\omega) = diag(\frac{1}{\sqrt{\pmb{deg}}}) \pmb{C}^*(\omega) $$

Define the Laplacian: $ L(\omega,\tau^v) = I - \alpha C(\omega,\tau^v)$, and $L(\omega, \tau^v)$ is an 86x86 complex matrix because we parcellated 86 brain regions.

Eigen decompose the Laplacian: $L(\alpha,k) = U(\alpha,k)\Lambda(\alpha,k)U^H(\alpha,k)$

We believe the eigenmodes $U(\alpha,k)$ are a substrate of the brain's white matter connections on which functional activity plays out.

$U(\alpha,k)$ is a vector whose numbers represent the magnitude of power in each parcellated brain region.

## What do these eigenmodes look like?

We compare our findings with the conventional non-complex structural connectome.

<center>
<img src = "images/eigmode_examples2.png" alt = "eig_examples" width = "1300" style = "align:center"/>
</center>

Eigenmodes of the complex Laplacian exhibit spatial patterns of canonical networks.

<center>
<img src = "images/fig3_rescaled_compiled.png" alt = "fc_eig" width = "1100"style = "align:center"/>
</center>

#### Spatial pattern matching of complex Laplacian outperforms real Laplacian:

<center>
<img src = "images/pearson_residual.png" alt = "pearson" width = "1200" style = "align:center"/>
</center>

### Next step: Examine the frequency spectrum of eigenmodes
<center>
<img src = "../assets/SGM_1.png" alt = "SGM1" width = "1200" style ="align:center"/>
</center>

<p style = "text-align:center; font-size:200%;"> Examine Frequency Spectrum with a Parsimonious Structure-Function Model </p>
<center>
<img src = "../assets/SGM1_model.png" alt = "sg1" width = "600" style ="align:center"/>
</center>

and the structural eigenmodes act as spatial filters!
<center>
    <img src = "images/intro_pysurfer.png" alt = "eigenmodes" width = "800" style = "align:center"/> 
</center>

Evaluating the eigen decomposition of the complex graph Laplacian: $L(\omega) = U(\omega)\Lambda(\omega)U^H(\omega)$, and considering $U(\omega)U^H(\omega) = I$, our spectral graph model is defined as:
$$ X(\omega) = \sum_{i} \frac{u_i(\omega) u_i^H (\omega)}{j\omega + \frac{1}{\tau_C} \lambda_i F_e(\omega)} H_{local}(\omega) \boldsymbol{P}(\omega) $$

Frequency spectrum depends on structural connectomes:
<center>
<img src = "../assets/SGM_fig2AB.png" alt = "sg3" width = "1000" style ="align:center"/>
</center>

Frequency spectrums for uniform and random connectomes:
<center>
<img src = "../assets/SGM_fig2CD.png" alt = "sg4" width = "1000" style ="align:center"/>
</center>

Introducing sparsity:
<center>
<img src = "../assets/SGM_fig2EF.png" alt = "sg5" width = "1000" style ="align:center"/>
</center>

<aside class="notes">
Frame this in actual neuro anatomy: if you have all to all connection or random connection that's filled up between your brain regions, you have no direction of your signal spread, your signal is just scattering all over the place randomly or all to all. But if there's sparsity like in actual humans, the signal spread is more directed according to your anatomy.
</aside>

#### Parameter inference with Markov Chain Monte Carlo simulations (MCMC):

- 15 Random walkers scan through the space of 5 global parameters
- Model will produce simulated frequency spectrum and try to match empirical MEG spectrum

<center>
<img src = "../assets/mcmc_animated.gif" alt = "mcmc_gif" width = "700" style ="align:center"/>
</center>

### Did our walkers find highly probable parameters that reproduce MEG spectrum?

<center>
<img src = "../assets/SGM_mcmc_params.png" alt = "mcmc" width = "1200" style ="align:center"/>
</center>

Group level comparison:
<center>
<img src = "../assets/SGM_ave_spectrum.png" alt = "sg7" width = "1000" style ="align:center"/>
</center>

Spatial Patterns in Alpha-band:

<center>
<img src = "../assets/SGM_alpha_spatial.png" alt = "sg7" width = "1100" style ="align:center"/>
</center>

Spatial Patterns in Beta-band:

<center>
<img src = "../assets/SGM_beta_spatial.png" alt = "sg7" width = "1100" style ="align:center"/>
</center>

Spectral graph model performance:
<center>
    <img src = "../assets/SGM_spatial_violin.png" alt = "model_comparison" width = "1200" style = "align:center"/>
</center>

## Take-aways
 - We extracted canonical functional patterns of brain activity via complex-valued structural eigenmodes.
 - MEG Alpha and beta band activity can be mapped to individual complex-valued structural eigenmodes.
 - Our model captures BOTH frequency patterns as well as spatial patterns of brain activity while being parsimonious.

Thank You For Listening
---

 - Former PostDoc Dr. Pablo Damasceno contributed a lot to this project, now a data scientist at UCSF's Center for Intelligence Imaging.
 - Dr. Srikantan Nagarajan (UCSF) is a close collaborator and contributed a lot to this project.
 - Check out the [open source repository for this project](https://github.com/Raj-Lab-UCSF/spectrome)
 - Stay tuned for the world's first [GPU based tractography library](https://www.protocols.io/private/57B53B66EBB411E9837D0242AC110002)!